<font size=5, font color='blue'> stata_df is the dataframe downloaded through Stata API

In [30]:
import pandas as pd
pd.options.mode.chained_assignment = None

stata_df = pd.read_stata("SAU_1.dta")
stata_df.head()

,period,period_start_day,value,obs_status,frequency,dataset_code,dataset_name,indexed_at,provider_code,series_code,series_name,series_num,freq,section
0,2000,2000-01-01,NA,,annual,EGDP2,Expenditure on Gross Domestic Product (at purc...,2022-04-04T17:33:39.545Z,SAMA,change-in-stock,Change in Stock,Master,,
1,2001,2001-01-01,NA,,annual,EGDP2,Expenditure on Gross Domestic Product (at purc...,2022-04-04T17:33:39.545Z,SAMA,change-in-stock,Change in Stock,Master,,
2,2002,2002-01-01,NA,,annual,EGDP2,Expenditure on Gross Domestic Product (at purc...,2022-04-04T17:33:39.545Z,SAMA,change-in-stock,Change in Stock,Master,,
3,2003,2003-01-01,32762.982242331243,,annual,EGDP2,Expenditure on Gross Domestic Product (at purc...,2022-04-04T17:33:39.545Z,SAMA,change-in-stock,Change in Stock,Master,,
4,2004,2004-01-01,18337.578758768595,,annual,EGDP2,Expenditure on Gross Domestic Product (at purc...,2022-04-04T17:33:39.545Z,SAMA,change-in-stock,Change in Stock,Master,,


In [33]:
stata_df[(stata_df['value']=='NA')|(stata_df['value']=='')].shape[0]

1905

<font size=5, font color='blue'> python_df is the dataframe downloaded through Python API. 

In [34]:
python_df = pd.read_pickle("SAU_1.pickle")
python_df.head()

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,freq,Frequency,section,Section
0,annual,SAMA,BPR,Balance of payments (millions riyals),1.A,Current account - Annual,2007,2007-01-01,349985.449634,349985.449634,A,Annual,NaN,NaN
1,annual,SAMA,BPR,Balance of payments (millions riyals),1.A,Current account - Annual,2008,2008-01-01,496208.294794,496208.294794,A,Annual,NaN,NaN
2,annual,SAMA,BPR,Balance of payments (millions riyals),1.A,Current account - Annual,2009,2009-01-01,78579.791,78579.791000,A,Annual,NaN,NaN
3,annual,SAMA,BPR,Balance of payments (millions riyals),1.A,Current account - Annual,2010,2010-01-01,250316.219906,250316.219906,A,Annual,NaN,NaN
4,annual,SAMA,BPR,Balance of payments (millions riyals),1.A,Current account - Annual,2011,2011-01-01,594544.564,594544.564000,A,Annual,NaN,NaN


In [35]:
python_df[(python_df['value']=='NA')|(python_df['value']=='')].shape[0]

0

<font size=5, font color='blue'> Check whether the two dataframes have the same number of rows, which means equal number of data points / observations. Column numbers of the two dataframes may differ due to the difference between Stata and Python APIs, and so do the order, datatypes and names of columns and the order of rows, which is why we don't require a complete match between python_df and stata_df.

In [36]:
stata_df.shape[0]==python_df.shape[0]

True

<font size=5, font color='blue'> ['period_start_day'] in stata_df is the most accurate timestamp assigned to each observation (row). In python_df its equivalent is ['period'], so we rename it to match the column name in stata_df. We match the datatype of the two columns by converting both of them to pandas datetime objects. We also convert the datatype of ['value'] in stata_df to numeric.

In [37]:
python_df.rename(columns={'period':'period_start_day'},inplace=True)
stata_df['period_start_day'] = pd.to_datetime(stata_df['period_start_day'])
python_df['period_start_day'] = pd.to_datetime(python_df['period_start_day'])
stata_df['value'] = pd.to_numeric(stata_df['value'],'coerce').astype('float64')
python_df['value'] = python_df['value'].astype('float64')

<font size=5, font color='blue'> Check whether ['period_start_day', 'series_code'] can identify a unique row in both stata_df and python_df, if not then there is a duplication or error in Dbnomics.

In [38]:
cols = ['period_start_day', 'series_code']
sum(stata_df.duplicated(subset=cols))+sum(python_df.duplicated(subset=cols)) == 0

False

<font size=5, font color='blue'> The provider reused series codes in several datasets, so we now need ['period_start_day','series_code,'dataset_code'] to identify a unique row.

In [39]:
sum(stata_df.duplicated(subset=cols))

168

In [40]:
sum(python_df.duplicated(subset=cols))

168

In [41]:
sum(stata_df.duplicated(subset=cols+['dataset_code']))+sum(python_df.duplicated(subset=cols+['dataset_code']))==0

True

<font size=5, font color='blue'> Then we sort the two dataframes by ['period_start_day', 'series_code','dataset_code']. This ensures observations in the two dataframes are in the same order. Finally we check whether the two sub-frames [['period_start_day','series_code','dataset_code','value']] sliced from stata_df and python_df are exactly the same.

In [42]:
stata_sub = stata_df.sort_values(by=cols+['dataset_code'])[cols+['dataset_code','value']].reset_index(drop=True)
python_sub = python_df.sort_values(by=cols+['dataset_code'])[cols+['dataset_code','value']].reset_index(drop=True)
stata_sub.equals(python_sub)

False

<font size=5, font color='blue'> Compare rows in stata_df without missing values with the corresponding rows in Python

In [66]:
stata_notna_index = stata_sub[stata_sub.notna().all(axis=1)].index

In [90]:
python_sub.loc[stata_notna_index].equals(stata_sub.loc[stata_notna_index])

False

<font size=5, font color='blue'> Upon closer inspection, the differences in ['value'] of stata_df and python_df are extremely small and can be ignored.

In [76]:
a = python_sub.iloc[stata_notna_index]

In [77]:
b = stata_sub.iloc[stata_notna_index]

In [99]:
(a['value'] != b['value']).sum()

17

In [78]:
diffs = a.compare(b)
diff_index_labels = diffs.index.unique().tolist()
print(diff_index_labels)

[112, 119, 211, 241, 245, 251, 374, 378, 385, 508, 510, 638, 1032, 1036, 1038, 1163, 1620]


In [100]:
b.loc[diff_index_labels]['value']-a.loc[diff_index_labels]['value']

112     5.684342e-14
119     5.820766e-11
211    -2.328306e-10
241     3.637979e-12
245     1.164153e-10
251     2.910383e-11
374    -2.328306e-10
378     5.820766e-11
385    -5.820766e-11
508    -1.164153e-10
510     5.820766e-11
638    -2.328306e-10
1032    1.455192e-11
1036   -1.164153e-10
1038    5.820766e-11
1163    1.818989e-12
1620    2.910383e-11
Name: value, dtype: float64